In [4]:
import gzip
import json
import pickle
import urllib
from collections import Set
from typing import MutableSet
import data
from data import load
import urllib.request
from tqdm import tqdm

prs = set()

def to_api_url(url: str):
    return url[:url.rindex("#")]\
        .replace("github.com/", "api.github.com/repos/")\
        .replace("/pull/", "/pulls/")

def to_issue_comments_url(api_url: str):
    return api_url.replace("pulls", "issues") + "/comments"

def to_comments_url(api_url: str):
    return api_url + "/comments"

# i = 0
# for l in tqdm(load(), total=25323640, desc="Records", unit='record'):
#     #print(l)
#     i += 1
#     if l["URL"] is not None:
#         prs.add(to_api_url(l["URL"]))
#         if '#discussion' not in (l["URL"] or "#discussion"):
#             print("Found: ", l)
#             break
# 
# pickle.dump(prs, gzip.open(data.prs_file, 'wb+', 9))

prs: MutableSet[str] = pickle.load(gzip.open(data.prs_file, 'rb', 9))

print(len(prs), "PRs")

pr_docs = []

pbar = tqdm(prs)
skipped = 0
for pr_url in pbar:
    try:
        repo, num = data.PR.repo_and_num_from_api_url(pr_url)
        pr_docs.append(data.PR.create_from_api(repo, num))
    except Exception as e:
        skipped  += 1
    
    pbar.set_postfix_str("Skipped: " + str(skipped))
    
    if len(pr_docs) == 10_000:
        break

pickle.dump(pr_docs, gzip.open(data.prs_obj_file, 'wb+', 9))


3863509 PRs



  0%|          | 68/3863509 [01:30<1735:36:01,  1.62s/it, Skipped: 9]

KeyboardInterrupt: 